In [1]:
from ast import literal_eval
import re
regex1 = re.compile(r"\(([a-zA-Z0-9]*)\)", re.IGNORECASE)
regex2 = re.compile(r"\"([a-zA-Z0-9]*)\"", re.IGNORECASE)

import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv("processed_data/movies.csv")

In [3]:
movies.columns

Index(['Unnamed: 0', 'genres', 'id', 'imdb_id', 'original_language',
       'overview', 'popularity', 'tagline', 'title', 'vote_average',
       'vote_count', 'year', 'description', 'movie_id', 'cast', 'keywords',
       'director'],
      dtype='object')

In [4]:
movies['description'] + movies['keywords'] + movies['director'] + movies['cast']

0      minions stuart, kevin and bob are recruited by...
1      an amazon princess comes to the world of man t...
2      a live-action adaptation of disney's version o...
3      after being coerced into working for a crime b...
4      the special bond that develops between plus-si...
                             ...                        
495    this third film in the series follows a group ...
496    as kevin flynn searches for proof that he inve...
497    in a gritty and alternate 1985 the glory days ...
498    a group of young journalists investigate a cul...
499    set in 1879 paris. an orphan girl dreams of be...
Length: 500, dtype: object

In [5]:
re.sub(r'(foo)', r' \g<1> 123', 'foobar')

' foo 123bar'

In [6]:
np.any(movies["keywords"].apply(literal_eval).apply(lambda x: [regex2.sub(r' \g<1>', regex1.sub(r" \g<1>", str.lower("".join(i.split(" "))).replace(".", "").replace("-", "").replace("'", ""))) for i in x]).apply(lambda x: np.any(['"' in i or '(' in i for i in x])))

False

In [7]:
def concat_list_data_to_single_dataframe(dataframe, column, new_column_name=None):
    new_column_name = new_column_name if new_column_name is not None else column
    serie = dataframe[column].apply(literal_eval).apply(lambda x: [regex2.sub(r' \g<1>', regex1.sub(r" \g<1>", str.lower("".join(i.split(" "))).replace(".", "").replace("-", "").replace("'", "").replace(",", ""))) for i in x])
    serie = serie.apply(lambda x: " ".join(x))
    serie_frame = pd.concat([pd.DataFrame(i, columns=[new_column_name]) 
                                for i in serie.apply(lambda x: [i for i in x.split(" ")])
                               ], ignore_index=True)
    serie_frame.drop(inplace=True, index=serie_frame.index[serie_frame[new_column_name].duplicated()])
    serie_frame.drop(inplace=True, index=serie_frame.index[serie_frame[new_column_name] == ""])
    return serie_frame

In [8]:
keywords_df = concat_list_data_to_single_dataframe(movies, "keywords", "keyword")
print(keywords_df.shape)

(2722, 1)


In [9]:
keywords_df

,keyword
0,assistant
1,aftercreditsstinger
2,duringcreditsstinger
3,evilmastermind
4,minions
...,...
5389,deathofsuperhero
5390,americanpresident
5404,opera
5409,statueofliberty


In [10]:
cast_df = concat_list_data_to_single_dataframe(movies, "cast")
print(cast_df.shape)
cast_df.cast[cast_df.cast.str.contains("julialouis")]

(1243, 1)


1854    julialouisdreyfus
Name: cast, dtype: object

In [11]:
cast_df

,cast
0,sandrabullock
1,jonhamm
2,michaelkeaton
3,allisonjanney
4,galgadot
...,...
1990,aaronashmore
1991,meghanheffern
1992,benlewis
1995,carlyraejepsen


In [12]:
directors = movies.director.apply(lambda x: str.lower("".join(x.split(" "))).replace(".", "").replace("-", "").replace("'", "").replace(",", ""))
directors.drop(inplace=True, index=directors.index[directors.duplicated()])
print(directors.shape)

(331,)


In [13]:
genres = concat_list_data_to_single_dataframe(movies, "genres", "genre")

In [14]:
print(genres.shape)
genres

(18, 1)


,genre
0,family
1,animation
2,adventure
3,comedy
4,action
6,fantasy
9,romance
11,crime
23,sciencefiction
29,thriller


In [15]:
df = pd.concat([genres.genre, directors, cast_df.cast, keywords_df.keyword], ignore_index=True)
df.name = 0
df

0                  family
1               animation
2               adventure
3                  comedy
4                  action
              ...        
4309     deathofsuperhero
4310    americanpresident
4311                opera
4312      statueofliberty
4313     eiffeltowerparis
Name: 0, Length: 4314, dtype: object

In [16]:
movies.keywords = movies.keywords.apply(literal_eval).apply(
                        lambda x: [str.lower("".join(i.split(" "))).replace(".", "").replace("-", "").replace("'", "") for i in x]
                    ).apply(lambda x: " ".join(x))
movies.keywords

0      assistant aftercreditsstinger duringcreditssti...
1      dccomics hero greekmythology island worldwari ...
2      france magic castle fairytale musical curse an...
3      robbery atlanta music crimeboss romance tinnit...
4      brotherbrotherrelationship hero talent revenge...
                             ...                        
495    diving dolphin florida sequel scubadiving swim...
496    videogame hacker virtualreality dystopia compu...
497    dccomics secretidentity massmurder retirement ...
498    journalist coffin diary village supernatural c...
499    paris flying inventor dance opera musical life...
Name: keywords, Length: 500, dtype: object

In [17]:
movies.cast = movies.cast.apply(literal_eval).apply(
                        lambda x: [str.lower("".join(i.split(" "))).replace(".", "").replace("-", "").replace("'", "").replace(",", "") for i in x]
                    ).apply(lambda x: " ".join(x))
movies.cast

0      sandrabullock jonhamm michaelkeaton allisonjanney
1             galgadot chrispine robinwright dannyhuston
2             emmawatson danstevens lukeevans kevinkline
3            anselelgort lilyjames kevinspacey jamiefoxx
4            scottadsit ryanpotter danielhenney tjmiller
                             ...                        
495    dennisquaid bessarmstrong simonmaccorkindale l...
496    jeffbridges bruceboxleitner davidwarner cindym...
497    malinåkerman billycrudup matthewgoode jackieea...
498     cindysampson aaronashmore meghanheffern benlewis
499    ellefanning danedehaan carlyraejepsen maddiezi...
Name: cast, Length: 500, dtype: object

In [18]:
movies.genres = movies.genres.apply(literal_eval).apply(
                        lambda x: [str.lower("".join(i.split(" "))) for i in x]
                    ).apply(lambda x: " ".join(x))
movies.genres

0             family animation adventure comedy
1                      action adventure fantasy
2                        family fantasy romance
3                                  action crime
4      adventure family animation action comedy
                         ...                   
495                             thriller horror
496             sciencefiction action adventure
497               action mystery sciencefiction
498                                      horror
499           animation family adventure comedy
Name: genres, Length: 500, dtype: object

In [19]:
movies.director = movies.director.apply(lambda x: str.lower("".join(x.split(" "))).replace(".", "").replace("-", "").replace("'", "").replace(",", ""))
movies.director

0            kylebalda
1         pattyjenkins
2           billcondon
3          edgarwright
4        chriswilliams
            ...       
495           joealves
496    stevenlisberger
497         zacksnyder
498          jonknautz
499         ericsummer
Name: director, Length: 500, dtype: object

In [20]:
concat = movies.director + " " + movies.genres + " " + movies.cast + " " + movies.keywords
concat

0      kylebalda family animation adventure comedy sa...
1      pattyjenkins action adventure fantasy galgadot...
2      billcondon family fantasy romance emmawatson d...
3      edgarwright action crime anselelgort lilyjames...
4      chriswilliams adventure family animation actio...
                             ...                        
495    joealves thriller horror dennisquaid bessarmst...
496    stevenlisberger sciencefiction action adventur...
497    zacksnyder action mystery sciencefiction malin...
498    jonknautz horror cindysampson aaronashmore meg...
499    ericsummer animation family adventure comedy e...
Length: 500, dtype: object

In [21]:
concat.str.contains("action").sum()

210

In [22]:
concat.str.contains("stevenspielbergstevenspielbergroyscheider").sum() # ancienne erreur

0

In [23]:
concat.str.contains("julialouisdreyfus").sum()

1

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [25]:
tf = TfidfVectorizer(analyzer='word', min_df=0, stop_words='english')
data_for_tfidf = movies.description
data_for_tfidf += " " + movies.director
data_for_tfidf += " " + movies.cast
data_for_tfidf += " " + movies.keywords
data_for_tfidf += " " + movies.genres
tfidf_matrix = tf.fit_transform(data_for_tfidf)

In [26]:
tfidf_matrix.shape

(500, 9234)

In [27]:
print(tfidf_matrix[0])

  (0, 1691)	0.08171965028918367
  (0, 227)	0.061727701541336014
  (0, 461)	0.09892929671202902
  (0, 2992)	0.07495309354992942
  (0, 90)	0.1049426870662529
  (0, 2849)	0.20335593271485133
  (0, 2558)	0.10920015931302984
  (0, 245)	0.11156886971267099
  (0, 619)	0.1907163597142094
  (0, 352)	0.20335593271485133
  (0, 5421)	0.1747923809720822
  (0, 4489)	0.1907163597142094
  (0, 7189)	0.1907163597142094
  (0, 4723)	0.1907163597142094
  (0, 1041)	0.20335593271485133
  (0, 715)	0.14750137418929252
  (0, 3828)	0.11999463628493012
  (0, 3588)	0.1747923809720822
  (0, 9138)	0.07815313382001708
  (0, 6320)	0.1450062010496624
  (0, 3699)	0.1907163597142094
  (0, 3775)	0.20335593271485133
  (0, 3956)	0.15646929395110898
  (0, 4197)	0.16430352431729803
  (0, 360)	0.20335593271485133
  (0, 8851)	0.14750137418929252
  (0, 8023)	0.15318488820962375
  (0, 6031)	0.20335593271485133
  (0, 7239)	0.1907163597142094
  (0, 6736)	0.16910886695175095
  (0, 1007)	0.1907163597142094
  (0, 4623)	0.1747923809720

In [28]:
tf.get_feature_names()[2000:6000]

['cultureclash',
 'cultures',
 'cunnilingus',
 'cup',
 'cupboard',
 'curator',
 'cure',
 'curiosity',
 'curious',
 'current',
 'currie',
 'curries',
 'curse',
 'cursed',
 'curt',
 'curtis',
 'custodian',
 'custody',
 'customers',
 'cut',
 'cutoffhead',
 'cutter',
 'cuttingthecord',
 'cyber',
 'cyberpunk',
 'cyborg',
 'cynthiagarris',
 'cypher',
 'd2',
 'da',
 'dacremontgomery',
 'dafnekeen',
 'dafoe',
 'dagen',
 'dagger',
 'daisyridley',
 'dakota',
 'dakotafanning',
 'dakotagoyo',
 'dakotajohnson',
 'dale',
 'daley',
 'dallasroberts',
 'damage',
 'damien',
 'damienchazelle',
 'damme',
 'damn',
 'damnation',
 'damned',
 'dan',
 'danaykroyd',
 'dance',
 'dancecontest',
 'danced',
 'dancer',
 'dancing',
 'danedehaan',
 'danfogelman',
 'danfogler',
 'danfutterman',
 'danger',
 'dangerous',
 'dangerously',
 'dangers',
 'dangilroy',
 'danhedaya',
 'daniel',
 'danielcraig',
 'danielespinosa',
 'danielhenney',
 'danielkaluuya',
 'daniellaalonso',
 'danielradcliffe',
 'danielschechter',
 'danie

In [29]:
tf.get_feature_names().index("stevenspielberg")

7895

In [30]:
tfidf_matrix[:,tf.get_feature_names().index("stevenspielberg")]

<500x1 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [31]:
genres_indices = []
for i in genres.genre:
    genres_indices.append(tf.get_feature_names().index(i))
print(genres_indices)
print(tfidf_matrix[0,genres_indices[0]])
tfidf_matrix[:,genres_indices] *= 5
print(tfidf_matrix[0,genres_indices[0]])

[2992, 461, 227, 1691, 170, 3006, 7043, 1941, 7260, 8332, 5681, 2498, 8941, 3894, 9025, 3828, 8592, 5661]
0.07495309354992942
0.3747654677496471


In [32]:
directors_indices = []
for i in directors:
    directors_indices.append(tf.get_feature_names().index(i))
# print(directors_indices)
print(tfidf_matrix[0,directors_indices[0]])
tfidf_matrix[:,directors_indices] *= 3
print(tfidf_matrix[0,directors_indices[0]])

0.1907163597142094
0.5721490791426281


In [33]:
keywords_indices = []
for i in keywords_df.keyword:
    try:
        keywords_indices.append(tf.get_feature_names().index(i))
    except:
        print(i)
# print(keywords_indices)
print(tfidf_matrix[0,keywords_indices[0]])
tfidf_matrix[:,keywords_indices] *= 10
print(tfidf_matrix[0,keywords_indices[0]])

fire
mine
0.1907163597142094
1.907163597142094


In [34]:
cast_indices = []
for i in cast_df.cast:
    try:
        cast_indices.append(tf.get_feature_names().index(i))
    except:
        print(i)
# print(cast_indices)
print(tfidf_matrix[0,cast_indices[0]])
tfidf_matrix[:,cast_indices] *= 3
print(tfidf_matrix[0,cast_indices[0]])

0.1907163597142094
0.5721490791426281


In [35]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [36]:
cosine_sim.shape

(500, 500)

In [37]:
cosine_sim[0]

array([1.00000000e+00, 9.08489607e-02, 1.17025043e-01, 0.00000000e+00,
       5.22275737e-01, 2.86241477e-01, 4.11685509e-01, 7.34902205e-02,
       0.00000000e+00, 0.00000000e+00, 8.76658139e-02, 1.23978354e-01,
       1.19420296e-01, 0.00000000e+00, 2.00685387e-01, 0.00000000e+00,
       0.00000000e+00, 1.37440871e-01, 4.73194781e-01, 0.00000000e+00,
       0.00000000e+00, 5.95968767e-05, 1.04788419e-02, 4.76698399e-05,
       1.03994356e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       5.65403413e-05, 0.00000000e+00, 5.46513806e-05, 1.33212520e-01,
       8.47518489e-02, 0.00000000e+00, 3.02297112e-01, 0.00000000e+00,
       7.03894774e-05, 1.13071057e-01, 1.26935133e-01, 7.35206942e-02,
       0.00000000e+00, 1.10119590e-01, 1.20109830e-02, 1.11880264e-01,
       9.73551663e-02, 7.02305639e-02, 1.99131303e-01, 2.22443533e-01,
       0.00000000e+00, 0.00000000e+00, 1.40212309e-01, 2.11825758e-01,
       3.71335013e-01, 0.00000000e+00, 2.34491024e-01, 2.56821949e-01,
      

In [38]:
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

In [39]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [40]:
movie='Star Wars'
print("Description of the Movie: ", movie)
print('---------------------------------------------------------------------')
print(movies[movies['title']==movie]['overview'])

Description of the Movie:  Star Wars
---------------------------------------------------------------------
45    Princess Leia is captured and held hostage by ...
Name: overview, dtype: object


In [41]:
movie='Toy Story'
print("Description of the Movie: ", movie)
print('---------------------------------------------------------------------')
print(movies[movies['title']==movie]['overview'])

Description of the Movie:  Toy Story
---------------------------------------------------------------------
206    Led by Woody, Andy's toys live happily in his ...
Name: overview, dtype: object


In [42]:
get_recommendations('Star Wars').head(20)

294         The Empire Strikes Back
96     Star Wars: The Force Awakens
239                Star Trek Beyond
61          Avengers: Age of Ultron
453    Independence Day: Resurgence
496                            Tron
67     Rogue One: A Star Wars Story
17                     The Avengers
31          Guardians of the Galaxy
185                   Power Rangers
115              Mad Max: Fury Road
136                         Ant-Man
218               Jupiter Ascending
222                    TRON: Legacy
319                      Iron Man 2
214                       Divergent
202                        Iron Man
430                  Fantastic Four
298                  Fantastic Four
358                       Allegiant
Name: title, dtype: object

In [43]:
get_recommendations('Toy Story').head(20)

456                                 Toy Story 3
480            Spirit: Stallion of the Cimarron
65                              Despicable Me 3
200                      Penguins of Madagascar
312                                      Cars 3
301                                          Up
160                             Despicable Me 2
415                                 Toy Story 2
231                              In a Heartbeat
382                                  Shark Tale
18     Captain Underpants: The First Epic Movie
84                              The Emoji Movie
171                                  Inside Out
455                    Smurfs: The Lost Village
126         Diary of a Wimpy Kid: The Long Haul
139                              Monsters, Inc.
464                                A Bug's Life
4                                    Big Hero 6
327                                        Cars
352                               Lilo & Stitch
Name: title, dtype: object